In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [69]:
# 2910 reconfinement
nn=1
JE,JI=1.60,1.2
a,b=0.0,0.0#1.5,0.25#0.3,0.2#
# a=-JI*(JE/b)#+0.8
# a=-JI/(JE/b)#-0.3

'''
J=np.zeros((nn*2,nn*2))
J[:,:nn],J[:,nn:]=JE/nn,-JI/nn
J[:nn,:nn],J[nn:,:nn]=J[:nn,:nn]-a/nn,J[nn:,:nn]+a/nn
J[:nn,nn:],J[nn:,nn:]=J[:nn,nn:]+b/nn,J[nn:,nn:]-b/nn
print('2by2 matrix:',J)
lvec,sv,rvech=la.svd(J)
print('sum:',np.sum(sv),'diff:',sv[0]-sv[1])
s1=np.sqrt(2*((JE+b)**2+(JI-a)**2))
s2=np.sqrt(2*((JE-b)**2+(JI+a)**2))
print('sum1:',s1,'; sum2:',s2)
'''

# generate perturbed matrix
nn=600
# # JE,JI=0.8,0.6
# # a,b=0.80,0.2#1.5,-0.3
# a1=-JE*b/JI
# a2=-JE*JI/b
# print('a1-',a1,' a2-',a2)
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
# Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
# Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
print(' random singular value:',svr[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
x_= x+Jr
lvec_,sv_,rvech_=la.svd(x_)
theo_sv=1/svr[:2]+svr[:2]
# print('theoretical solution:',theo_sv,'; numerical solution:',sv_[:2])

rholimit=1/svr[0]+svr[0]
derivphi=1/(1-svr[0]**2)
overlap=-1/derivphi/svr[0]**2
# print('am theo:',1/sv[0]**2)

print('overlap num:',np.sum(lvec_[:,0]*lvec[:,0])**2)




# Tomorrow, check whether the overlap of the perturbed matrix is equal to the original * alpha square.

 random singular value: [2.82842712e+00 2.51212613e-14]
overlap num: 0.8761283878410749


In [74]:
## calculate whether there is an additive relationship
overlap_num=1.0-1/svr[0]**2
alpha = np.sqrt(overlap_num)
rvec_,rvec=rvech_.T,rvech.T
lvar,rvar=np.zeros_like(lvec_[:,0]),np.zeros_like(rvec_[:,0])
lvar[:nn]=lvec_[:nn,0]-alpha*lvec[:nn,0]
lvar[nn:]=lvec_[nn:,0]-alpha*lvec[nn:,0]
print('overlap:',alpha**2)
print('sigma:',rholimit,'; numsigmas:',sv_[0])
print('1/sigma:',1/sv_[0]**2)
plt.figure()
n, bins, patches = plt.hist(lvar[:], 100, density=True, facecolor='g', alpha=0.75)


rvar[:nn]=rvec_[:nn,0]-alpha*rvec[:nn,0]
rvar[nn:]=rvec_[nn:,0]-alpha*rvec[nn:,0]
plt.figure()
n, bins, patches = plt.hist(rvar[:], 100, density=True, facecolor='g', alpha=0.75)
plt.figure()
n, bins, patches = plt.hist(rvec_[:,0], 100, density=True, facecolor='g', alpha=0.75)
print(np.mean(np.sum(lvar*rvar)))
# plt.figure()
# n, bins, patches = plt.hist(lvar[:nn], 100, density=True, facecolor='g', alpha=0.75)
print(np.mean(np.sum(lvar[:nn]*rvar[:nn])))
# plt.figure()
# n, bins, patches = plt.hist(lvar[nn:], 100, density=True, facecolor='g', alpha=0.75)
print(np.mean(np.sum(lvar[nn:]*rvar[nn:])))
print('rvector:',rvec[:2,0],'; ',rvec[nn:nn+2,0])
print('tilte rvector:',rvec_[:10,0],'; ',rvec_[nn:nn+10,0])
#tilte overlap
tilteuv=np.sum(rvec_[:,0]*lvec_[:,0])
orguv=alpha**2*np.sum(rvec[:,0]*lvec[:,0])
cosuv=(1/svr[0]**2)*np.sum(rvec[:,0]*lvec[:,0])

overlap: 0.8749999999999996
sigma: 3.1819805153394594 ; numsigmas: 3.202035340065009
1/sigma: 0.09753214112167677
-0.0021557554204781606
0.0008231584202601689
-0.0029789138407383293
tilte rvector: [-0.03265986 -0.03265986] ;  [0.0244949 0.0244949]
tilte rvector: [-0.02735731 -0.03549696 -0.04522799 -0.03124508 -0.03046251 -0.03884344
 -0.01234629 -0.02786855 -0.05734784 -0.01903706] ;  [0.00585008 0.0067866  0.03829516 0.02833079 0.01775737 0.015754
 0.01489371 0.01419926 0.00404347 0.02658142]


** What's the difference between substract tilde and original **

In [9]:
# 2910 reconfinement
nn=1
JE,JI=2.60,1.4
a,b=0.5,1.25#0.3,0.2#
# generate perturbed matrix
nn=60
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]+a/nn,Jr[nn:,:nn]-a/nn
Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
print(' random singular value:',svr[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
print('determinant of random matrix:',la.det(x))
x_= x+Jr
lvec_,sv_,rvech_=la.svd(x_)
theo_sv=1/svr[:2]+svr[:2]

## calculate whether there is an additive relationship
overlap_num=1.0-1/svr[0:2]**2
alpha = np.sqrt(overlap_num)
print('overlap:',alpha**2,1.0-1/svr[0:2]**2)

rvec_,rvec=rvech_.T,rvech.T
lvar,rvar=np.zeros_like(lvec_[:,0:2]),np.zeros_like(rvec_[:,0:2])

lvar[:nn]=lvec_[:nn,0]-alpha*lvec[:nn,0]
lvar[nn:]=lvec_[nn:,0]-alpha*lvec[nn:,0]
rvar[:nn]=rvec_[:nn,0]-alpha*rvec[:nn,0]
rvar[nn:]=rvec_[nn:,0]-alpha*rvec[nn:,0]

subtilde=x_-sv_[0]*np.reshape(lvec[:,0],(nn*2,1))@np.reshape(rvech[0,:],(1,nn*2))
determinantsubt=la.det(subtilde)
print('determinant of matrix, substract tilde:',determinantsubt)
rhop=1/svr[0]+svr[0]
lvecrec,rvecrec=alpha*lvec[:,0],alpha*rvec[:,0]
suborg=x_-rhop*np.reshape(lvecrec,(2*nn,1))@np.reshape(rvecrec,(1,nn*2))
determinantsubo=la.det(suborg)
print('determinant of matrix, substract original:',determinantsubo)
stlvec,stsv,strvech=la.svd(subtilde)
solvec,sosv,sorvech=la.svd(suborg)
plt.figure()
plt.plot(stsv,'r')
plt.plot(sosv,'b')

# plt.figure()
# n, bins, patches = plt.hist(lvar[:], 100, density=True, facecolor='g', alpha=0.75)
# plt.figure()
# n, bins, patches = plt.hist(rvar[:], 100, density=True, facecolor='g', alpha=0.75)
# plt.figure()
# n, bins, patches = plt.hist(rvec_[:,0], 100, density=True, facecolor='g', alpha=0.75)


 random singular value: [4.18310712 1.88854834]
determinant of random matrix: 1.59002349462521e-26
overlap: 0.9428518694675886 0.9428518694675887
determinant of matrix, substract tilde: 1.7210522036633843e-26
determinant of matrix, substract original: 1.9802324928387983e-26


Contents
* Can the perturbed matrix $\widetilde{X}$ be represented by basis vectors of low-rank matrix?
* Will the difference between different representations change with the properties of low-rank structure/random matrix (infimum/supramum)
* Can we use either of the (M,N)/(m,n) to represent the perturbed matrix?
* Now we use E/I structure (known already), how about change to use Gaussian distribution?

Edit by Yuxiu

30/10/2020 the $1^{st}$ version V0


In [3]:
# limit of the singular value of the perturbed matrix
# find the largest singular value which are larger than the critical value.
thetac = 2.0
rlarger=svr[np.where(svr>thetac)]
lenc = len(rlarger)
rhop=np.zeros(lenc)
x_tilde_rec=np.zeros_like(x_)
# reconstruct perturbed matrix using basis vectors of low-rank structure
for i in range(lenc):
    rhop[i]=1/svr[i]+svr[i]
    vecoverlap=1-1/svr[i]**2
    lvecrec=np.sqrt(vecoverlap)*lvec[:,i]
    rvecrec=np.sqrt(vecoverlap)*rvec[:,i]
    x_tilde_rec +=rhop[i]*np.reshape(lvecrec,(2*nn,1))@np.reshape(rvecrec,(1,nn*2))#rhop[i]*(lvecrec@rvecrec.T)
print(lenc)


errmat=x_-x_tilde_rec#(rhop[0]*np.reshape(lvecrec,(nn*2,1))@np.reshape(rvecrec,(1,nn*2)))#(sv_[0]*np.reshape(lvec_[:,0],(nn*2,1))@np.reshape(rvec_[:,0].T,(1,nn*2)))
# check the singular value of the Error-matrix
lvecrec,svrec,rvechrec=la.svd(x_tilde_rec)
lvecerr,sverr,rvecherr=la.svd(errmat)
print('recon svrho:',svrec[:5],'; num svrho:',svr[:5])
print('num vec overlap:',np.sum(rvec_[:,0]*rvec[:,0])**2,' theo overlap:',vecoverlap)

print('singular values (largest) of the error matrix:',sverr[:5],sv_[:5])
fig=plt.figure()
ax1=fig.add_subplot(121)
n, bins, patches = ax.hist(sverr, 50, density=True, facecolor='g', alpha=0.75)
ax2=fig.add_subplot(122)
n, bins, patches = ax.hist(sverr, 50, density=True, facecolor='g', alpha=0.75)


1
recon svrho: [6.80859064e+00 6.63107729e-14 6.31349745e-14 5.42914578e-14
 5.31285661e-14] ; num svrho: [6.81175455e+00 5.88095309e-14 5.73088770e-14 4.43769051e-14
 4.24557572e-14]
num vec overlap: 0.9791361252922339  theo overlap: 0.9784482758620688
singular values (largest) of the error matrix: [1.98660329 1.97867277 1.97106312 1.9663761  1.95873549] [6.9618902  1.98562974 1.97727465 1.96883838 1.96626655]


In [4]:
# check whether the Generalized SVD can identically represent the perturbed matrix (all the properties)
# we start from rank-1 perturbation 
m,n=np.ones((nn*2,1)),np.zeros((nn*2,1))
n[:nn,0],n[nn:,0]=JE,-JI
newrep=(1-1/svr[0]**4)*m@n.T/nn
diff_rep=newrep-x_tilde_rec
# plt.imshow(diff_rep)
newrandMat=x_-newrep
lvecnr,svnr,rvechnr=la.svd(newrandMat)
fig=plt.figure()
ax=fig.add_subplot(121)
n, bins, patches = ax.hist(svnr, 50, density=True, facecolor='g', alpha=0.75)

lvecr,svr,rvechr=la.svd(x)
ax1=fig.add_subplot(122)
n, bins, patches = ax1.hist(svr, 50, density=True, facecolor='g', alpha=0.75)

In [5]:
# some simple analysis, how rho_1 changes with JE and JI, how the overlap of vectors changes with JE and JI


In [389]:
# Generate Gaussian vector of E neuron
JEmI,JEpI=1.4,4.4
nn,a,b=1000,0.6,-0.9
yE,yI= np.random.randn(nn),np.random.randn(nn)


JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
print(JE,JI)
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE*(1+yE)/nn,-JI*(1+yI)/nn
# M1,N1=np.ones(2*nn),np.ones(2*nn)
# N1[:nn],N1[nn:]=JE*(1+0)/nn,-JI*(1+0)/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=yE,-yI
N2[:nn],N2[nn:]=a*(1+yE)/nn,b*(1+yI)/nn
# M2[:nn],M2[nn:]=1,-1
# N2[:nn],N2[nn:]=a*(1+0)/nn,b*(1+0)/nn
print(np.mean(a*yE),np.mean(b*yI))

print('overlap M2N2:',np.sum(M2*N2),'; M1N2:',np.sum(M1*N2))

# Jr=np.zeros((nn*2,nn*2))
# Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
# Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
# Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
# lvec,svr,rvech=la.svd(Jr)


2.9000000000000004 1.5000000000000002
0.0017747510472494853 -0.043191081287862525
overlap M2N2: 1.5336838938170185 ; M1N2: -0.341416330240613


In [225]:
J = np.reshape(M1,(2*nn,1))@np.reshape(N1,(1,2*nn))
J +=np.reshape(M2,(2*nn,1))@np.reshape(N2,(1,2*nn))
print(np.shape(J))

(2000, 2000)


In [235]:
# Generate Gaussian vector of E neuron
JEmI,JEpI=-0.5,2.4
nn,a,b=100,0.6,-0.6
yE,yI= np.random.randn(nn),np.random.randn(nn)


JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
print(JE,JI)
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=yE,-yI
N2[:nn],N2[nn:]=a*yE/nn,b*yI/nn
print(np.mean(a*yE),np.mean(b*yI))

print('overlap M2N2:',np.sum(M2*N2),'; M1N2:',np.sum(M1*N2))

# Jr=np.zeros((nn*2,nn*2))
# Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
# Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
# Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
# lvec,svr,rvech=la.svd(Jr)


0.95 1.45
-0.05453256133680691 0.09585319211219419
overlap M2N2: 1.2997835767445287 ; M1N2: 2.0816681711721685e-17


In [390]:
J = np.reshape(M1,(2*nn,1))@np.reshape(N1,(1,2*nn))
J +=np.reshape(M2,(2*nn,1))@np.reshape(N2,(1,2*nn))
x =0.00*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
J += x
Jovl=Nrank.T@Mrank
Jovl[0,0],Jovl[0,1],Jovl[1,0],Jovl[1,1]=np.sum(M1*N1),np.sum(N1*M2),np.sum(N2*M1),np.sum(N2*M2)
print(np.shape(J))

(2000, 2000)


In [391]:
# find out the singular values of the matrix
lvec,sigv,rvech=la.svd(J)
theo_sigv=np.zeros(2)
lvec,sigvo,revech=la.svd(Jovl)
l1,l2=np.sqrt((JE+b)**2+(JI+a)**2)/np.sqrt(2),np.sqrt((JE-b)**2+(JI-a)**2)/np.sqrt(2)
theo_sigv[0],theo_sigv[1]=(l1+l2),np.abs(l1-l2)
print('singular value numerical:',sigv[:3],', theoretical:',theo_sigv,sigvo)

singular value numerical: [6.81295832e+00 1.02433605e+00 3.12575244e-15] , theoretical: [4.81194992 0.71073059] [4.87630568 0.73398404]


In [395]:
def tanhpfunc(x):
    return np.tanh(x)+1.0
def dynxact(J,x0,timeparams,type='tanhp'):
    tfinal,dt=timeparams[0],timeparams[1]
    nn=np.shape(J)[0]
    nt = int(tfinal/dt)
    xactdyn=np.zeros((nn,nt))
    # print(xFP)
    xactdyn[:,0]=x0
    eps=1e-5
    if type=='tanhp':
        for it in range(1,nt):
            phisyn=np.reshape(tanhpfunc(xactdyn[:,it-1]),(nn,1))
            deltaxact=dt*(-xactdyn[:,it-1]+np.squeeze(J@phisyn))
            xactdyn[:,it]=xactdyn[:,it-1]+deltaxact
    return xactdyn
def dynkappa(M,N,kappa0,timeparams,type='tanhp',express='EI'):
    tfinal,dt=timeparams[0],timeparams[1]
    nn=np.shape(M)[0]
    nt = int(tfinal/dt)
    kappadyn=np.zeros((2,nt))
    kappadyn[:,0]=kappa0
    xactdyn = np.zeros((nn,nt))
    xactdyn[:,0]=kappadyn[0,0]*M[:,0]+kappadyn[1,0]*M[:,1]
    eps=1e-5
    if type=='tanhp':
        for it in range(1,nt):
            xact = kappadyn[0,it-1]*M[:,0]+kappadyn[1,it-1]*M[:,1]
            phisyn=np.reshape(tanhpfunc(xact),(nn,1))
            deltakappa=dt*(-kappadyn[:,it-1]+np.squeeze(N.T@phisyn))
            kappadyn[:,it]=kappadyn[:,it-1]+np.squeeze(deltakappa)
            xactdyn[:,it]=kappadyn[0,it]*M[:,0]+kappadyn[1,it]*M[:,1]
    elif type=='tanh':
        for it in range(1,nt):
            xact = kappadyn[0,it-1]*M[:,0]+kappadyn[1,it-1]*M[:,1]
            phisyn=np.reshape(np.tanh(xact),(nn,1))
            deltakappa=dt*(-kappadyn[:,it-1]+np.squeeze(N.T@phisyn))
            kappadyn[:,it]=kappadyn[:,it-1]+np.squeeze(deltakappa)
            xactdyn[:,it]=kappadyn[0,it]*M[:,0]+kappadyn[1,it]*M[:,1]

    return (kappadyn,xactdyn)
timeparams=np.array([30,0.1])
x0=-1*np.random.rand(nn*2)
# x0[nn:]-=1.0
xact=dynxact(J,x0,timeparams,type='tanhp')
print('axticity:',xact[:3,-1],xact[nn:nn+3,-1])


axticity: [-1.02270675 -0.69384884 -0.67600542] [-1.61324275 -2.29687409 -1.98023333]


In [396]:
plt.figure()
tt = np.linspace(0,timeparams[0],int(timeparams[0]/timeparams[1]))
for i in range(30):
    plt.plot(tt,xact[i,:],'r')
    plt.plot(tt,xact[nn+i,:],'b')
frE,frI = xact[:nn,-1],xact[nn:,-1]
fig=plt.figure()
ax=fig.add_subplot(222)
n, bins, patches = ax.hist(frE, 50, density=True)#, facecolor='g', alpha=0.75)
print('mean xE:',np.mean(frE))
ax1=fig.add_subplot(224)
n, bins, patches = ax1.hist(frI, 50, density=True)#, facecolor='g', alpha=0.75)
print('mean xI:',np.mean(frI))
ax=fig.add_subplot(223)
n, bins, patches = ax2.hist((frE-frI)/2.0, 50, density=True)#, facecolor='g', alpha=0.75)
print('mean xI:',np.mean((frE-frI)/2.0))

mean xE: -0.9148048889090731
mean xI: -0.8832480898810284
mean xI: -0.015778399514022295


In [397]:
# All functions used below
def derivtransfer(FPs,type='tanhp'):
    xe,xi=FPs[0],FPs[1]
    if type=='tanhp':
        derive=1/np.cosh(xe)**2
        derivi=1/np.cosh(xi)**2
        return np.array([derive,derivi])
# give J mat 
Jder=np.array([[JE-JI,JE+JI],[a+b,a-b]])
eigv,eigvec=la.eig(Jder)
# calculate the derivative of the fixed point
FPs=np.array([np.mean(frE),np.mean(frI)])
derx=derivtransfer(FPs,'tanhp')
print('reciprocal:',1/derx,eigv)

reciprocal: [2.09798268 2.00531245] [1.45+1.14782403j 1.45-1.14782403j]


In [257]:
R=2
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
kappa0=1.0*np.random.rand(2)
kappadyn,xactdyn=dynkappa(Mrank,Nrank,kappa0,timeparams,type='tanhp',express='EI')
tt = np.linspace(0,timeparams[0],int(timeparams[0]/timeparams[1]))
# plt.figure()
# for i in range(30):
#     plt.plot(tt,xact[i,:],'r')
#     plt.plot(tt,xact[nn+i,:],'b')
frE,frI = xactdyn[:nn,-1],xactdyn[nn:,-1]
# fig=plt.figure()
# ax1=fig.add_subplot(211)
# ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(frE, 50, density=True, alpha=0.5)#, facecolor='g', alpha=0.75)
print('mean xE:',np.mean(frE))
n, bins, patches = ax2.hist(frI, 50, density=True, alpha=0.5)#, facecolor='g', alpha=0.75)
print('mean xI:',np.mean(frI))
# ax=fig.add_subplot(223)
# n, bins, patches = ax2.hist((frE-frI)/2.0, 50, density=True)#, facecolor='g', alpha=0.75)
# print('mean xI:',np.mean((frE-frI)/2.0))

mean xE: 0.4195529275321085
mean xI: 0.4215750937902644


In [193]:
# ax1.set_title(r'$J_E=2.9$'+str(JE)+r',$J_I=1.5$'+str(JI)+',$a=$'+str(a)+',$b=$'+str(b))
ax1.set_title(r'$J_E=2.9$'+', '+r',$J_I=1.5$'+',$a=$'+str(a)+',$b=$'+str(b))
ax1.set_xlabel(r'$x_E^*$',fontsize=14)
ax2.set_xlabel(r'$x_I^*$',fontsize=14)

Text(0.5, 47.04444444444444, '$x_I^*$')

In [258]:
# tt = np.linspace(0,timeparams[0],int(timeparams[0]/timeparams[1]))
# fig=plt.figure()
# ax3=fig.add_subplot(111)
# ax3.plot(tt,kappadyn[0,:],'r',linewidth=1.5,label=r'$\kappa_1^{EI}$')
# ax3.plot(tt,kappadyn[1,:],'b',linewidth=1.5,label=r'$\kappa_2^{EI}$')
# ax3.set_title(r'$J_E=1.2$'+r',$J_I=$'+str(JI)+',$a=$'+str(a)+',$b=$'+str(b))
# ax3.set_xlabel('time',fontsize=14)
# ax3.set_ylabel(r'$\kappa^{EI}$'+' dynamics',fontsize=14)
# plt.grid()
# plt.legend()
# plt.show()

ax3.plot(tt,kappadyn[0,:],'r--',linewidth=1.5)#,label=r'$\kappa_1^{EI}$')
ax3.plot(tt,kappadyn[1,:],'b--',linewidth=1.5)#,label=r'$\kappa_2^{EI}$')

** Check whether there is a rotation between using EI and singular vectors **

In [5]:
nn=1000
JE,JI,a,b=2.0,1.2,0.2,-1.2
Jr = np.zeros((2*nn,2*nn))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]+a/nn,Jr[nn:,:nn]-a/nn
Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,sigv,rvech=la.svd(Jr)
print('singular values:',sigv[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
x_= x+Jr

R = 2
lvecr,sigvr,rvechr=la.svd(x_)
u2td = np.reshape(lvecr[:,:R],(nn*2,R))
v2td = np.reshape(rvechr[:R,:],(R,nn*2))
v2td = v2td.T
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=1,-1
N2[:nn],N2[nn:]=a/nn,b/nn
R=1
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2

# printent to be

fsig=np.zeros((R,R))
alpha2,sigvout=np.zeros(R),np.zeros(R)
for i in range(R):
    alpha2[i] = (1-1/sigv[i]**2)
    sigvout[i]=1.0/sigv[i]+sigv[i]
    fsig[i,i]=sigvout[i]
u2 = np.reshape(lvec[:,:R],(nn*2,R))
v2 = np.reshape(rvech[:R,:],(R,nn*2))
v2 = v2.T
alpha = np.zeros((R,R))
for i in range(R):
    for j in range(R):
        alpha[i,j]=np.sum(u2td[:,i]*u2[:,j])
print('random left vec1:',alpha[0,0]**2+alpha[0,1]**2,alpha2[0],alpha[0,0],alpha[0,1])
print('random left vec2:',alpha[1,0]**2+alpha[1,1]**2,alpha2[1],alpha[1,0],alpha[1,1])
print('zero product:',alpha[0,0]*alpha[1,0]+alpha[0,1]*alpha[1,1])
u2c=u2.copy()
u2c[:,0]=alpha[0,0]*u2[:,0]#+alpha[0,1]*u2[:,1]
u2c[:,1]=alpha[1,1]*u2[:,1]#+alpha[1,0]*u2[:,0]

v2c=v2.copy()
u2c[:,0]=alpha[0,0]*v2[:,0]#+alpha[0,1]*v2[:,1]
v2c[:,1]=alpha[1,1]*v2[:,1]#+alpha[1,0]*v2[:,0]

x_svdre = u2c@fsig@v2c.T
lvec_re,sigv_re,rvech_re=la.svd(x_svdre)
print('reconstructed sv:',sigv_re[:R])
print('limit rho:',sigvout)
print('random sv:',sigvr[:R])

# print('leftvector:',u2td.T@u2)
# print('rightvector:',v2td.T@v2)
# print('alpha:',alpha)

singular values: [3.29848450e+00 4.83692517e-14]
random left vec1: 0.9052963541542657 0.9080882352941175 0.9514705273707227 0.00043554465264880257
random left vec2: 0.0005358636218363978 -4.2742632162187084e+26 -0.005882100829494745 0.02238893726080037
zero product: -0.005586894196384699
reconstructed sv: [1.03632885e+10 3.42666748e+00]
limit rho: [3.60165403e+00 2.06742913e+13]
random sv: [3.59611551 1.99085598]


In [48]:
for i in range(R):
    for j in range(R):
        alpha[i,j]=np.sum(v2td[:,i]*v2[:,j])
print('random left vec1:',alpha[0,0]**2+alpha[0,1]**2,alpha2[0],alpha[0,0],alpha[0,1])
print('random left vec2:',alpha[1,0]**2+alpha[1,1]**2,alpha2[1],alpha[1,0],alpha[1,1])
print('zero product:',alpha[0,0]*alpha[1,0]+alpha[0,1]*alpha[1,1])
x_svdre = u2@fsig@v2.T
lvec_re,sigv_re,rvech_re=la.svd(x_svdre)
print('reconstructed sv:',sigv_re[:R])
print('limit rho:',sigvout)
print('random sv:',sigvr[:R])

random left vec1: 0.9207269924350023 0.918870170929377 -0.9594651247412663 -0.012396242988442885
random left vec2: 0.33384296183066653 0.3395317495095759 -0.010720594557017404 0.5776919859951414
zero product: 0.0031248263630773276
reconstructed sv: [0. 0.]
limit rho: [3.79566233 2.04317047]
random sv: [3.78875126 2.03338448]


In [143]:
# 2910 reconfinement
nn=1
JE,JI=2.60,1.4
a,b=0.5,1.25#0.3,0.2#
# generate perturbed matrix
nn=600
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]+a/nn,Jr[nn:,:nn]-a/nn
Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
print(' random singular value:',svr[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
print('determinant of random matrix:',la.det(x))
x_= x+Jr
lvec_,sv_,rvech_=la.svd(x_)
theo_sv=1/svr[:2]+svr[:2]

## calculate whether there is an additive relationship
overlap_num=1.0-1/svr[0:2]**2
alpha = np.sqrt(overlap_num)
print('overlap:',alpha**2,1.0-1/svr[0:2]**2)

rvec_,rvec=rvech_.T,rvech.T
thetac = 1.0
rlarger=svr[np.where(svr>thetac)]
lenc = len(rlarger)
rhop=np.zeros(lenc)
x_tilde_rec=np.zeros_like(x_)
lvecrec,rvecrec=np.zeros_like(lvec),np.zeros_like(rvec)
# reconstruct perturbed matrix using basis vectors of low-rank structure
for i in range(lenc):
    rhop[i]=1/svr[i]+svr[i]
    vecoverlap=1-1/svr[i]**2
    lvecrec[:,i]=np.sqrt(vecoverlap)*lvec[:,i]+1/svr[i]*np.random.randn(2*nn)/np.sqrt(nn*2.0)
    print('left norm:',np.sum(lvecrec[:,i]**2))
    rvecrec[:,i]=np.sqrt(vecoverlap)*rvec[:,i]+1/svr[i]*np.random.randn(2*nn)/np.sqrt(2.0*nn)
    print('right norm:',np.sum(rvecrec[:,i]**2))
    x_tilde_rec +=rhop[i]*np.reshape(lvecrec[:,i],(2*nn,1))@np.reshape(rvecrec[:,i],(1,nn*2))#rhop[i]*(lvecrec@rvecrec.T)
    # x_tilde_rec +=rhop[i]/svr[i]**2*(np.random.randn(2*nn,1)@np.random.randn(1,2*nn))/(nn*2)
print(lenc)


errmat=x_-x_tilde_rec#(rhop[0]*np.reshape(lvecrec,(nn*2,1))@np.reshape(rvecrec,(1,nn*2)))#(sv_[0]*np.reshape(lvec_[:,0],(nn*2,1))@np.reshape(rvec_[:,0].T,(1,nn*2)))
# check the singular value of the Error-matrix
lvecrec,svrec,rvechrec=la.svd(x_tilde_rec)
lvecerr,sverr,rvecherr=la.svd(errmat)
print('recon svrho:',svrec[:2],'; num svrho:',sv_[:2])
print('sig-sig^-3:',svr[:2]-svr[:2]**(-3))
print('num vec overlap:',np.sum(rvec_[:,0]*rvec[:,0])**2,' theo overlap:',vecoverlap)





 random singular value: [4.18310712 1.88854834]
determinant of random matrix: -1.2469162524996052e-262
overlap: [0.94285187 0.71962209] [0.94285187 0.71962209]
left norm: 1.024516816517132
right norm: 1.0072755550949255
left norm: 1.008405639691008
right norm: 0.9776364065928194
2
recon svrho: [4.49185816 2.40040639] ; num svrho: [4.41059929 2.43166288]
sig-sig^-3: [4.16944547 1.74008621]
num vec overlap: 0.9416338021688803  theo overlap: 0.71962209303842


In [150]:
# change the representations
# rotation matrix 2*2
# 2910 reconfinement
# nn=1
# JE,JI=2.60,1.4
# a,b=0.5,0.25#0.3,0.2#
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M2 = np.array([[1,1],[1,-1]])
ROT = np.sqrt(2)*lvec2@la.inv(M2)
print('Rotation:',ROT)
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=1,-1
N2[:nn],N2[nn:]=a/nn,b/nn

R=2
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
# test 
urec = Mrank@ROT.T/np.sqrt(2*nn)
print('compare u and u test:',urec[:2,:])
print('compare u and u theo:',lvec[:2,0],lvec[:2,1])
alphamat=np.zeros((R,R))
invsvr =np.zeros((R,R))
for i in range(2):
    alphamat[i,i]=np.sqrt(1-1/svr[i]**2)
    invsvr[i,i]=1/svr[i]
vtrec = np.sqrt(2*nn)*(invsvr@ROT@Nrank.T)#alphamat@
print('compare vt and vt test:',vtrec[:,:2])
print('compare vtand vt theo:',rvech[0,:2],rvech[1,:2])

Rotation: [[-0.9979024   0.06473634]
 [-0.06473634 -0.9979024 ]]
compare u and u test: [[-0.02693818 -0.03067574]
 [-0.02693818 -0.03067574]]
compare u and u theo: [-0.02693818 -0.02693818] [-0.03067574 -0.03067574]
compare vt and vt test: [[-0.03536305 -0.03536305]
 [-0.02039906 -0.02039906]]
compare vtand vt theo: [-0.03536305 -0.03536305] [-0.02039906 -0.02039906]


In [114]:
# TEST the random distribution 
residlvec,residrvec=np.zeros((nn*2,2)),np.zeros((nn*2,2))
crosslvec,crossrvec=np.zeros((nn*2,2)),np.zeros((nn*2,2))
rvec_=rvech_.T
for i in range(1):
    residlvec[:,i]=lvec_[:,i]+lvecrec[:,i]
    residrvec[:,i]=rvec_[:,i]+rvecrec[:,i]
    crosslvec[:,i]=lvec[:,i]*residlvec[:,i]
    crossrvec[:,i]=rvec[:,i]*residrvec[:,i]

for i in range(1,2):
    residlvec[:,i]=lvec_[:,i]-lvecrec[:,i]
    residrvec[:,i]=rvec_[:,i]-rvecrec[:,i]
    crosslvec[:,i]=lvec[:,i]*residlvec[:,i]
    crossrvec[:,i]=rvec[:,i]*residrvec[:,i]

# distribution of the residual
print('the first rank:')
print('mean:',np.mean(residlvec[:,1]),'; std:',np.std(residlvec[:,1]),'; theo:',1/svr[1])

the first rank:
mean: 0.0006157789126623077 ; std: 0.015356090443686764 ; theo: 0.5295072303203989


In [115]:
fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(crossrvec[:nn,0], 50, density=True, alpha=0.5)
n, bins, patches = ax2.hist(crossrvec[nn:,0], 50, density=True, alpha=0.5)

fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(residrvec[:nn,0], 50, density=True, alpha=0.5)
n, bins, patches = ax2.hist(residrvec[nn:,0], 50, density=True, alpha=0.5)

In [96]:
# print(np.sum(crossrvec[:,0])*2*np.sqrt(overlap_num[0]),np.mean(residrvec[:nn,0]),np.mean(residrvec[nn:,0]),np.sum(residrvec[:,0]**2),overlap_num[0])

print(np.sum(crossrvec[:,0])*1,np.sum(residrvec[:nn,0]**2)+np.sum(residrvec[nn:,0]**2))
# print((np.sqrt(overlap_num[0])+np.sum(crossrvec[:,0]))**2)
print('std:',np.std(residrvec[:,0])**2*nn*2)
print((overlap_num[0]),np.sum(crossrvec[:,0]),np.sqrt(overlap_num[0])),
idxc=0
print('sum should be:',0*np.sum(crossrvec[:,idxc])*2*np.sqrt(overlap_num[idxc])+overlap_num[idxc]+0*np.sum(residrvec[:,idxc]**2))
print((rvech_[0,:]@rvec[:,0])**2,overlap_num[0])

-0.00044817929308526606 0.056277761328291515
std: 0.056238581071729334
0.9428518694675888 -0.00044817929308526606 0.9710055970320608
sum should be: 0.9428518694675888
0.9437224395363871 0.9428518694675888


In [90]:
sqrEI,sqrab=JE**2+JI**2,a**2+b**2
# ((JI-a)**2+(JE-b)**2)*((JI+a)**2+(JE+b)**2)
lsqrt,abEI=np.sqrt(((JI-a)**2+(JE-b)**2)*((JI+a)**2+(JE+b)**2)),-a*JE+b*JI

u1ratio=(sqrEI-sqrab-lsqrt-2*abEI)/(sqrEI-sqrab-lsqrt+2*abEI)
print('ratio u1:',u1ratio)
print('mean ratio:',np.mean(residlvec[nn:,0])/np.mean(residlvec[:nn,0]))


ratio u1: -1.1387455915269222
mean ratio: 1.814155517736223


In [116]:
residmat=np.zeros((2,nn*2,nn*2))
for i in range(2):
    residmat[i,:,:]=np.reshape(residlvec[:,i],(nn*2,1))@np.reshape(residrvec[:,i],(1,nn*2))

flatresidmat=np.zeros((2,(nn*2)**2))
for i in range(2):
    flatresidmat[i,:]=np.reshape(residmat[i,:,:],(1,-1))
# fig=plt.figure()
# ax1=fig.add_subplot(111)
# ax1.scatter(flatresidmat[0,:],flatresidmat[1,:])

fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(residrvec[:,0], 100, density=True, alpha=0.5)
stdtest=np.std(residrvec[:,0])
xxx=np.linspace(-3*stdtest,3*stdtest,1000)
disttest = np.exp(-(xxx/stdtest)**2/2)/np.sqrt(2*np.pi*stdtest**2)
# disttest/=np.sum(disttest)
ax1.plot(xxx,disttest*nn*2/1000)

n, bins, patches = ax2.hist(residrvec[:,1], 100, density=True, alpha=0.5)
stdtest=np.std(residrvec[:,1])
xxx=np.linspace(-3*stdtest,3*stdtest,1000)
disttest = np.exp(-(xxx/stdtest)**2/2)/np.sqrt(2*np.pi*stdtest**2)
# disttest/=np.sum(disttest)
ax2.plot(xxx,disttest*nn*2/1000)

In [128]:
# 2910 reconfinement
nn=1
JE,JI=2.60,1.4
a,b=0.5,0.25#0.3,0.2#
# generate perturbed matrix
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]+a/nn,Jr[nn:,:nn]-a/nn
Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
M = np.array([[1,1],[1,-1]])
Mrr = np.sqrt(2)*la.inv(M)@lvec#M/np.sqrt(2)@la.inv(lvec)
Mrl = np.sqrt(2)*lvec@la.inv(M)
print('Rotation:',Mrr)
print('Rotation:',Mrl)
print(np.sum(Mrr[:,0]*Mrr[:,1]))
print(np.sum(Mrr[0,0]**2+Mrr[1,0]**2),np.sum(Mrr[0,1]**2+Mrr[1,1]**2))

print(np.sum(Mrl[:,0]*Mrl[:,1]))
print(np.sum(Mrl[0,0]**2+Mrl[1,0]**2),np.sum(Mrl[0,1]**2+Mrl[1,1]**2))

Rotation: [[-0.99383167  0.11089907]
 [-0.11089907 -0.99383167]]
Rotation: [[-0.99383167 -0.11089907]
 [ 0.11089907 -0.99383167]]
0.0
0.9999999999999998 0.9999999999999998
0.0
0.9999999999999998 0.9999999999999998


In [125]:
print(lvec)

[[-0.65984805 -0.75139906]
 [-0.75139906  0.65984805]]
